# An Example

This example demonstrates the process of loading dataset, training, evaluation and prediction, using an expamplary dataset. The dataset contains 500 graphene structures that are generated using Rebo potential. The code has been run with Tensorflow 2.6.

In [1]:
import atomdnn

# 'float32' is used for reading data and train by default, one can set data_type to 'float64' here
atomdnn.data_type = 'float64'

# force and stress are evaluated by default, 
# if one only need to compute potential energy, then set compute_force to false
atomdnn.compute_force = True

# default value is for converting ev/A^3 to GPa
# note that: the predicted positive stress means tension and negative stress means compression
stress_unit_convert = 160.2176 

import numpy as np
import tensorflow as tf
from atomdnn import data
from atomdnn import network
from atomdnn.data import Data
from atomdnn.data import *
from atomdnn.network import Network

In [2]:
# load tensorflow dataset, for Tensorflow version lower than 2.6, need to specify element_spec.
# The process of reading data and creating dataset is discussed in 'Data pipeline' section.

dataset = tf.data.experimental.load('example_tfdataset')

2021-10-18 19:38:30.072748: W tensorflow/stream_executor/platform/default/dso_loader.cc:65] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-10-18 19:38:30.072780: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-18 19:38:30.072833: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist


In [3]:
# split the data to training, validation and testing sets

train_dataset, val_dataset, test_dataset = split_dataset(dataset,0.7,0.2,0.1,shuffle=True)

Traning data: 350 images
Validation data: 100 images
Test data: 50 images


In [4]:
# Build the network
# See section 'Training' for detailed description on Network object.

elements = ['C']
act_fun = 'relu' # activation function
nfp = get_fingerprints_num(dataset) # number of fingerprints (or descriptors)
arch = [30,30] # NN layers

model = Network(elements = elements,\
                num_fingerprints = nfp,\
                arch = arch,\
                activation_function = act_fun)

In [5]:
# Train the model 

opt = 'Adam' # optimizer
loss_fun = 'rmse' # loss function
scaling = 'std' # scaling the traning data with standardization
lr = 0.02 # learning rate
loss_weights = {'pe' : 1, 'force' : 1, 'stress': 0.1} # the weights in loss function

model.train(train_dataset, val_dataset, \
            optimizer=opt, \
            loss_fun = loss_fun, \
            batch_size=30, \
            lr=lr, \
            epochs=30, \
            scaling=scaling, \
            loss_weights=loss_weights, \
            compute_all_loss=True, \
            shuffle=True, \
            append_loss=True)

2021-10-18 19:38:30.138312: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Forces are used for training.
Stresses are used for training.
Scaling factors are computed using training dataset.
Training dataset are standardized.
Validation dataset are standardized.
Training dataset will be shuffled during training.

===> Epoch 1/30 - 0.823s/epoch
     training_loss    - pe_loss: 329.196 - force_loss: 1186.271 - stress_loss: 51101.325 - total_loss: 6625.600
     validation_loss  - pe_loss: 71.290 - force_loss: 593.548 - stress_loss: 25557.508 - total_loss: 3220.589

===> Epoch 2/30 - 0.827s/epoch
     training_loss    - pe_loss: 64.465 - force_loss: 589.883 - stress_loss: 21025.056 - total_loss: 2756.854
     validation_loss  - pe_loss: 44.031 - force_loss: 524.868 - stress_loss: 20366.036 - total_loss: 2605.503

===> Epoch 3/30 - 0.800s/epoch
     training_loss    - pe_loss: 40.991 - force_loss: 415.016 - stress_loss: 14697.165 - total_loss: 1925.723
     validation_loss  - pe_loss: 19.335 - force_loss: 370.859 - stress_loss: 15252.499 - total_loss: 1915.444

===

In [14]:
# Evaluate using the first 5 data in test dataset

model.evaluate(test_dataset.take(5),return_prediction=False)

Evaluation loss is:
        pe_loss:       1.0684e+00
     force_loss:       6.7686e+00
    stress_loss:       4.2568e+01
     total_loss:       1.2094e+01
The total loss is computed using the loss weights - pe: 1.00 - force: 1.00 - stress: 0.10


In [7]:
# prediction using the first 5 data in test dataset

input_dict = get_input_dict(test_dataset.take(5))
model.predict(input_dict)

{'pe': array([-28.308469  , -30.70238078, -28.37588111, -28.59157203,
        -27.23017633]),
 'force': array([[[ -4.87540508,  16.20537386,  51.23653551],
         [  9.31678774,  -5.65669885, -31.3836977 ],
         [-13.68639882,   1.94890128, -20.72748745],
         [  9.24501616, -12.49757641,   0.87464961]],
 
        [[ 29.56516981,   5.88600637,  13.2147016 ],
         [ 19.65903927,  -3.42711633, -22.04439536],
         [-24.48530241,  -4.51321415,  14.96612353],
         [-24.73890695,   2.05432393,  -6.13642973]],
 
        [[ -9.5599586 ,   1.67236397,  14.88963643],
         [ -1.77675048,   3.96959366,  -1.45979697],
         [  9.60788002,  12.11863339, -14.99421279],
         [  1.7288291 , -17.76059094,   1.56437333]],
 
        [[  5.63561615,   6.51593179,  15.67414723],
         [  2.46443364,   4.54883307, -62.83720616],
         [ 11.02615808,  -5.37788488,  -4.08509065],
         [-19.12620769,  -5.68688008,  51.24814955]],
 
        [[-11.32141102, -36.32980774,

In [8]:
# save the trained model

descriptor = {'name': 'acsf', 
              'cutoff': 6.5001,
              'etaG2':[0.01,0.025,0.05,0.075,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.8,1,1.5,2,3,5,10], 
              'etaG4': [0.01], 
              'zeta': [0.08,0.1,0.15,0.2,0.3,0.35,0.5,0.6,0.8,1.,1.5,2.,3.0,4.,5.5,7.0,10.0,25.0,50.0,100.0],
              'lambda': [1.0, -1.0]}

save_dir = 'example.tfdnn'
network.save(model,save_dir,descriptor=descriptor)

2021-10-18 19:38:55.069516: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: example.tfdnn/assets
Network signatures and descriptor are written to example.tfdnn/parameters for LAMMPS simulation.


In [9]:
# print the tensorflow signature of the saved model

network.print_signature(save_dir)

The given SavedModel SignatureDef contains the following input(s):
  inputs['atom_type'] tensor_info:
      dtype: DT_INT32
      shape: (-1, -1)
      name: serving_default_atom_type:0
  inputs['center_atom_id'] tensor_info:
      dtype: DT_INT32
      shape: (-1, -1)
      name: serving_default_center_atom_id:0
  inputs['dgdr'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1, -1, -1, -1)
      name: serving_default_dgdr:0
  inputs['fingerprints'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1, -1, -1)
      name: serving_default_fingerprints:0
  inputs['neighbor_atom_coord'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1, -1, -1, -1)
      name: serving_default_neighbor_atom_coord:0
  inputs['neighbor_atom_id'] tensor_info:
      dtype: DT_INT32
      shape: (-1, -1)
      name: serving_default_neighbor_atom_id:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['atom_pe'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1, -1, 1)
      

In [10]:
# Load the saved model for continuous training and prediction

imported_model = network.load(save_dir)

Network has been inflated! self.built: True


In [11]:
# Re-train the model 

loss_weights = {'pe' : 1, 'force' : 1, 'stress': 0.1}

opt = 'Adam'
loss_fun = 'rmse'
scaling = 'std'

model.train(train_dataset, val_dataset, \
            optimizer=opt, \
            loss_fun = loss_fun, \
            batch_size=30, \
            lr=0.02, \
            epochs=5, \
            scaling=scaling, \
            loss_weights=loss_weights, \
            compute_all_loss=True, \
            shuffle=True, \
            append_loss=True)

Forces are used for training.
Stresses are used for training.
Scaling factors are computed using training dataset.
Training dataset are standardized.
Validation dataset are standardized.
Training dataset will be shuffled during training.

===> Epoch 1/5 - 0.809s/epoch
     training_loss    - pe_loss: 7.996 - force_loss: 33.511 - stress_loss: 768.810 - total_loss: 118.388
     validation_loss  - pe_loss: 7.985 - force_loss: 27.101 - stress_loss: 633.255 - total_loss: 98.412

===> Epoch 2/5 - 0.781s/epoch
     training_loss    - pe_loss: 4.084 - force_loss: 19.212 - stress_loss: 357.137 - total_loss: 59.009
     validation_loss  - pe_loss: 2.469 - force_loss: 15.713 - stress_loss: 196.805 - total_loss: 37.862

===> Epoch 3/5 - 0.799s/epoch
     training_loss    - pe_loss: 2.855 - force_loss: 12.243 - stress_loss: 149.484 - total_loss: 30.047
     validation_loss  - pe_loss: 1.552 - force_loss: 11.170 - stress_loss: 138.195 - total_loss: 26.541

===> Epoch 4/5 - 0.790s/epoch
     training

In [15]:
imported_model.evaluate(test_dataset.take(5),return_prediction=False)

Evaluation loss is:
        pe_loss:       1.1408e+00
     force_loss:       2.0264e+01
    stress_loss:       5.7532e+01
     total_loss:       2.7158e+01
The total loss is computed using the loss weights - pe: 1.00 - force: 1.00 - stress: 0.10


In [13]:
input_dict = get_input_dict(test_dataset.take(5))
imported_model.predict(input_dict)

{'pe': array([-28.308469  , -30.70238078, -28.37588111, -28.59157203,
        -27.23017633]),
 'force': array([[[ -4.87540508,  16.20537386,  51.23653551],
         [  9.31678774,  -5.65669885, -31.3836977 ],
         [-13.68639882,   1.94890128, -20.72748745],
         [  9.24501616, -12.49757641,   0.87464961]],
 
        [[ 29.56516981,   5.88600637,  13.2147016 ],
         [ 19.65903927,  -3.42711633, -22.04439536],
         [-24.48530241,  -4.51321415,  14.96612353],
         [-24.73890695,   2.05432393,  -6.13642973]],
 
        [[ -9.5599586 ,   1.67236397,  14.88963643],
         [ -1.77675048,   3.96959366,  -1.45979697],
         [  9.60788002,  12.11863339, -14.99421279],
         [  1.7288291 , -17.76059094,   1.56437333]],
 
        [[  5.63561615,   6.51593179,  15.67414723],
         [  2.46443364,   4.54883307, -62.83720616],
         [ 11.02615808,  -5.37788488,  -4.08509065],
         [-19.12620769,  -5.68688008,  51.24814955]],
 
        [[-11.32141102, -36.32980774,